In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [44]:
%%writefile rock_agent.py

# Обозначения для ходов:
# 0 - камень
# 1 - бумага
# 2 - ножницы

# 1. Агент, который всегда выбирает "камень"
def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


In [45]:
%%writefile paper_agent.py
# 2. Агент, который всегда выбирает "бумагу"
def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [46]:
%%writefile scissors_agent.py
# 3. Агент, который всегда выбирает "ножницы"
def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


In [47]:
%%writefile copy_opponent_agent.py
# 4. Агент, который копирует последний ход оппонента
import random

def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction  # копирует предыдущий ход противника
    else:
        return random.randrange(0, configuration.signs)  # случайный ход на первом шаге

Writing copy_opponent_agent.py


In [48]:
%%writefile anti_copy_agent.py
# 5. Агент пытается предугадать, что противник скопирует его прошлый ход,
# и выбирает ход, который победит этот предполагаемый выбор.
import random

def anti_copy_agent(observation, configuration):
    if observation.step > 0:
        predicted_opponent_action = (observation.lastOpponentAction + 1) % configuration.signs
        return (predicted_opponent_action + 1) % configuration.signs  # выбирает ход, который побеждает предполагаемый выбор противника
    else:
        return random.randrange(0, configuration.signs)  # случайный ход на первом шаге

Writing anti_copy_agent.py


In [49]:
%%writefile rock_paper_agent.py
# 6. Агент, который случайно выбирает между "камнем" и "бумагой"
import random

def rock_paper_agent(observation, configuration):
    return random.randint(0, 1)

Writing rock_paper_agent.py


In [50]:
%%writefile paper_scissors_agent.py
# 7. Агент, который случайно выбирает между "бумагой" и "ножницами"
import random

def paper_scissors_agent(observation, configuration):
    return random.randint(1, 2)

Writing paper_scissors_agent.py


In [ ]:
%%writefile rock_scissors_agent.py
# 8. Агент, который случайно выбирает между "камнем", "ножницами" и "бумагой"
import random

def rock_scissors_agent(observation, configuration):
    return random.randint(0, 2)

Writing rock_scissors_agent.py


In [52]:
%%writefile cycle_agent.py
# 9. Агент циклически повторяет ходы: камень -> ножницы -> бумага.

def cycle_agent(observation, configuration):
    if observation.step == 0:
        return 0  # камень на первом шаге
    else:
        return (observation.step % 3)  # циклический выбор: камень -> ножницы -> бумага

Writing cycle_agent.py


In [53]:
%%writefile reactionary_agent.py
# 10. Агент, который адаптировует свою стратегию в зависимости от того, какие действия выбирает противник.
# Реагирует на последнее действие противника, пытаясь выбрать действие, которое победит его.
import random
import math

# Функция get_score предназначена для вычисления результата между двумя действиями агентов.
# Возвращает 0 для ничьей, 1 для выигрыша первого игрока и -1 для выигрыша второго игрока.
def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

last_react_action = None
def reactionary_agent(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Writing reactionary_agent.py


In [54]:
%%writefile counter_reactionary_agent.py
# 11. Агент, который предугадывает действия противника, основываясь на результатах предыдущих раундов.

import random
import math

# Функция get_score предназначена для вычисления результата между двумя действиями агентов.
# Возвращает 0 для ничьей, 1 для выигрыша первого игрока и -1 для выигрыша второго игрока.
def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

def counter_reactionary_agent(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action


Writing counter_reactionary_agent.py


In [55]:
%%writefile statistical_agent.py

# 12. Агент, который анализирует действия противника и выбирает свое следующее действие на основе наиболее часто встречающегося (модального) действия противника. 
def statistical_agent(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical_agent.py


In [62]:
%%writefile pattern_agent.py
#13. Агент пытается найти повторяющийся шаблон в ходах противника и выбирает ход, который побеждает следующий ход в этом шаблоне.
import random

opponent_history = []

def pattern_agent(observation, configuration):
    global opponent_history
    
    if observation.step == 0:
        opponent_history = []  # Инициализация истории на первом шаге
        return random.randrange(0, configuration.signs)  # Случайный ход на первом шаге
    else:
        opponent_history.append(observation.lastOpponentAction)
        
        if len(opponent_history) < 3:
            return (opponent_history[-1] + 1) % configuration.signs  # Выбор хода, который побеждает последний ход противника
        
        # Поиск шаблона
        pattern_length = 3  # Длина шаблона для поиска
        if len(opponent_history) >= pattern_length:
            # Проверяем, есть ли повторяющийся шаблон
            for i in range(len(opponent_history) - pattern_length):
                if opponent_history[i:i + pattern_length] == opponent_history[-pattern_length:]:
                    predicted_opponent_action = opponent_history[(i + pattern_length) % len(opponent_history)]
                    return (predicted_opponent_action + 1) % configuration.signs  # Выбор хода, который побеждает предполагаемый следующий ход
        
        # Если не найден шаблон, выбираем ход, который побеждает последний ход противника
        return (opponent_history[-1] + 1) % configuration.signs


Overwriting pattern_agent.py


In [74]:
# Турнир между агентами
agents = ['anti_copy_agent', 'copy_opponent_agent', 'counter_reactionary_agent',
          'cycle_agent', 'paper_agent', 'paper_scissors_agent', 'pattern_agent',
          'reactionary_agent', 'rock_agent', 'rock_paper_agent', 'rock_scissors_agent',
          'scissors_agent', 'statistical_agent']

# Добавляем .py к каждому агенту
agents_py = [i + ".py" for i in agents]
n_agents = len(agents)

# Инициализируем матрицу результатов
scores = np.zeros((n_agents, n_agents))

# Играем каждый агент против каждого
for i in range(n_agents):
    for j in range(i + 1, n_agents):
        result = evaluate("rps", [agents_py[i], agents_py[j]], configuration={"episodeSteps": 100}, num_episodes=1)
        scores[i][j] = result[0][0]  # сохраняем результат для агента i
        scores[j][i] = result[0][1]  # сохраняем результат для агента j

# Определяем победителя
vic = []
for i in range(n_agents):
    cnt_vic = sum(1 for j in range(n_agents) if scores[i][j] > 0)  # считаем победы агента
    vic.append(cnt_vic)

# Определяем индексы победителей
i_victors = [i for i, v in enumerate(vic) if v == max(vic)]
if len(i_victors) > 1:
    print("The winners are", '\n', *[agents_py[i] for i in i_victors], "!")
else:
    print("The winner is", agents_py[i_victors[0]], "!")

The winner is pattern_agent.py !


In [75]:
# Построение матрицы корреляций
df_scores = pd.DataFrame(scores, index=agents, columns=agents)
df_scores

,anti_copy_agent,copy_opponent_agent,counter_reactionary_agent,cycle_agent,paper_agent,paper_scissors_agent,pattern_agent,reactionary_agent,rock_agent,rock_paper_agent,rock_scissors_agent,scissors_agent,statistical_agent
anti_copy_agent,0.0,-49.0,49.0,98.0,-97.0,-29.0,-99.0,49.0,-98.0,-30.0,0.0,-98.0,-96.0
copy_opponent_agent,49.0,0.0,99.0,-99.0,0.0,0.0,-64.0,-49.0,0.0,0.0,0.0,0.0,-20.0
counter_reactionary_agent,-49.0,-99.0,0.0,0.0,48.0,0.0,-44.0,0.0,49.0,0.0,0.0,48.0,0.0
cycle_agent,-98.0,99.0,0.0,0.0,0.0,0.0,-92.0,0.0,0.0,0.0,0.0,0.0,0.0
paper_agent,97.0,0.0,-48.0,0.0,0.0,-39.0,-99.0,-97.0,99.0,41.0,0.0,-99.0,-97.0
paper_scissors_agent,29.0,0.0,0.0,0.0,39.0,0.0,-37.0,0.0,0.0,29.0,0.0,-45.0,0.0
pattern_agent,99.0,64.0,44.0,92.0,99.0,37.0,0.0,29.0,97.0,0.0,0.0,98.0,27.0
reactionary_agent,-49.0,49.0,0.0,0.0,97.0,0.0,-29.0,0.0,98.0,29.0,-20.0,97.0,57.0
rock_agent,98.0,0.0,-49.0,0.0,-99.0,0.0,-97.0,-98.0,0.0,-55.0,0.0,99.0,-98.0
rock_paper_agent,30.0,0.0,0.0,0.0,-41.0,-29.0,0.0,-29.0,55.0,0.0,0.0,0.0,0.0
